<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [1]:
#!pip3 install twilio

     ---------------------------------------- 1.4/1.4 MB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 KB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 97.1/97.1 KB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 KB ? eta 0:00:00
     -------------------------------------- 140.9/140.9 KB 8.7 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 KB 9.7 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\Jiord\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip3 install tqdm

     ---------------------------------------- 77.1/77.1 KB 2.1 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\Jiord\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [36]:
query = 'Dublín'
api_key = API_KEY_WAPI
days = 4

url_clima = f'http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={query}&days={days}&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=e270aea42e49482db0d204832232103&q=Dublín&days=4&aqi=no&alerts=no'

In [37]:
response = requests.get(url_clima).json()

In [38]:
response

{'location': {'name': 'Dublín',
  'region': 'Dublin',
  'country': 'Irlanda',
  'lat': 53.33,
  'lon': -6.25,
  'tz_id': 'Europe/Dublin',
  'localtime_epoch': 1679434195,
  'localtime': '2023-03-21 21:29'},
 'current': {'last_updated_epoch': 1679433300,
  'last_updated': '2023-03-21 21:15',
  'temp_c': 11.0,
  'temp_f': 51.8,
  'is_day': 0,
  'condition': {'text': 'Light rain',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/296.png',
   'code': 1183},
  'wind_mph': 18.6,
  'wind_kph': 29.9,
  'wind_degree': 180,
  'wind_dir': 'S',
  'pressure_mb': 989.0,
  'pressure_in': 29.21,
  'precip_mm': 3.1,
  'precip_in': 0.12,
  'humidity': 87,
  'cloud': 75,
  'feelslike_c': 7.9,
  'feelslike_f': 46.2,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 24.8,
  'gust_kph': 40.0},
 'forecast': {'forecastday': [{'date': '2023-03-21',
    'date_epoch': 1679356800,
    'day': {'maxtemp_c': 12.3,
     'maxtemp_f': 54.1,
     'mintemp_c': 8.5,
     'mintemp_f': 47.3,
     'avgtemp

In [39]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [40]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [41]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [42]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1679356800,
 'time': '2023-03-21 00:00',
 'temp_c': 9.4,
 'temp_f': 48.9,
 'is_day': 0,
 'condition': {'text': 'Patchy rain possible',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png',
  'code': 1063},
 'wind_mph': 7.8,
 'wind_kph': 12.6,
 'wind_degree': 188,
 'wind_dir': 'S',
 'pressure_mb': 1003.0,
 'pressure_in': 29.62,
 'precip_mm': 0.1,
 'precip_in': 0.0,
 'humidity': 93,
 'cloud': 58,
 'feelslike_c': 7.5,
 'feelslike_f': 45.5,
 'windchill_c': 7.5,
 'windchill_f': 45.5,
 'heatindex_c': 9.4,
 'heatindex_f': 48.9,
 'dewpoint_c': 8.2,
 'dewpoint_f': 46.8,
 'will_it_rain': 1,
 'chance_of_rain': 73,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 13.0,
 'gust_kph': 20.9,
 'uv': 1.0}

In [43]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2023-03-21'

In [44]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [45]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

1

In [46]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [47]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [48]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<?, ?it/s]


In [49]:
datos[0]

('2023-03-21', 0, 'Patchy rain possible', 9.4, 1, 73)

In [50]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-03-21,0,Patchy rain possible,9.4,1,73
1,2023-03-21,1,Overcast,9.7,0,0
2,2023-03-21,2,Overcast,9.8,0,0
3,2023-03-21,3,Patchy rain possible,9.1,0,65
4,2023-03-21,4,Overcast,8.6,0,0
5,2023-03-21,5,Patchy rain possible,8.5,0,61
6,2023-03-21,6,Light drizzle,8.9,1,81
7,2023-03-21,7,Patchy rain possible,9.0,1,79
8,2023-03-21,8,Cloudy,9.1,0,0
9,2023-03-21,9,Patchy rain possible,9.7,1,72


In [29]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1386357829.py, line 3)

In [51]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-03-21,0,Patchy rain possible,9.4,1,73
6,2023-03-21,6,Light drizzle,8.9,1,81
7,2023-03-21,7,Patchy rain possible,9.0,1,79
9,2023-03-21,9,Patchy rain possible,9.7,1,72
15,2023-03-21,15,Light drizzle,10.1,1,81
17,2023-03-21,17,Light drizzle,10.5,1,76
21,2023-03-21,21,Moderate rain,10.7,1,82


In [52]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [53]:
df['Fecha'][0]

'2023-03-21'

In [54]:
df_rain

,Condicion
Hora,
7,Patchy rain possible
9,Patchy rain possible
15,Light drizzle
17,Light drizzle
21,Moderate rain


In [55]:
f"""Hola!
El pronostico del tiempo hoy {df['Fecha'][0]} en {query} es: 
{str(df_rain)}
"""

'Hola!\nEl pronostico del tiempo hoy 2023-03-21 en Dublín es: \n                 Condicion\nHora                      \n7     Patchy rain possible\n9     Patchy rain possible\n15           Light drizzle\n17           Light drizzle\n21           Moderate rain\n'

In [56]:
PHONE_NUMBER

'+1 517 878 9044'

# Mensaje SMS desde Twilio

In [57]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+523111285778'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM760794f298d0c51ec8189a20a340355b


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />